# Hierarchical Indexing (Multi-indexing)

While Pandas does provide Panel and Panel4D objects that natively handle three-dimensional and four-dimensional data (see Aside: Panel Data), a far more common pattern in practice is to make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index. In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In [2]:
import pandas as pd
import numpy as np

## A mutiple indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional Series. For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way

Suppose you would like to track data about states from two different years. Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [6]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

### The better way: Pandas multiIndex

Pandas MultiIndex type give us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

In [9]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Notice that the MultiIndex contains multiple levels of indexing-in this case, the state names and the years, as well as multiple labels for each data point which encode these levels.
If we re-index our series with this MultiIndex, we see the hierachical representation of the data:

In [10]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the Series representation show the multiple index values, while the third column shows the data. Notice that some entries are missing in the first column: in this multi-indexing representation, any blank entry indicates the same value as the line above it.<br><br>
Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [11]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in. The syntax is much more convenient (and the operation is much more efficient) than the home-spun tuple-based multi-indexing solution that we started with. 

### MultiIndex as extra dimension

For the example above, we could easily have stored the same data using a simple DataFrame with index and column label. In fact, Pandas is built with this equivalence in mind, The unstack() method will quickly convert a mutliply indexed Series into a conventionally indexed DataFrame:

In [13]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the stack() method provides the opposite operation：

In [15]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all. The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional Series, we can also use it to **represent data of three or more dimensions in a Series or DataFrame**. Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a MultiIndex this is as easy as adding another column to the DataFrame:

In [16]:
pop_df = pd.DataFrame({'total': pop, 
                      'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In addition, all the ufuncs and other functionality discussed in Operating on Data in Pandas work with hierarchical indices as well. Here we compute the fraction of people under 18 by year, given the above data:

In [19]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex creation

The most straightforward way to construct a multiply indexed Series or DataFrame is to simply pass a list of two or more index arrays to the constructor.

In [21]:
df = pd.DataFrame(np.random.rand(4, 2), 
                 index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns = ['data1', 'data2'])
df

data1     data2
a 1  0.628366  0.955888
  2  0.809172  0.855533
b 1  0.008216  0.801645
  2  0.986145  0.557905


The work of creating the MultiIndex is done in the background. <br>

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a MultiIndex by default:

In [23]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a MultiIndex; we'll see a couple of these methods here.

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the pd.MultiIndex. For example, as we did before, you can construct the MultiIndex from a simple list of arrays giving the index values within each level:

In [29]:
index_mt = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
index_mt

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [26]:
df = pd.DataFrame(np.random.rand(4, 2), 
                 index = index_mt,
                 columns = ['data1', 'data2'])
df

data1     data2
a 1  0.827569  0.660173
  2  0.190922  0.423902
b 1  0.989120  0.702385
  2  0.304752  0.609631

You can construct it from a list of tuples giving the multiple index values of each point:

In [27]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can even construct it from a Cartesian product of single indices:

In [30]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

All above are equivalent. Thus, we can consider only use one.

### MultiIndex level names

Sometimes it is convenient to name the levels of the MultiIndex. This can be accomplished by passing the names argument to any of the above MultiIndex constructors, or by setting the names attribute of the index after the fact:

In [35]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

With more involved datasets, this can be a useful way to keep track of the meaning of various index value

### MultiIndex for columns

In a DataFrame, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well. Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [36]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  37.7  21.0  35.9  29.0  36.0
     2      29.0  37.2  42.0  36.5  54.0  36.9
2014 1      53.0  37.5  45.0  36.4  28.0  36.2
     2      15.0  36.8  29.0  36.0  33.0  38.3

In [65]:
# try again using the from_arrays method
index = pd.MultiIndex.from_arrays([[2013, 2013, 2014, 2014],[1, 2, 1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_arrays([['Bob', 'Bob','Guido','Guido', 'Sue', 'Sue'], ['HR', 'Temp', 'HR', 'Temp', 'HR', 'Temp']],
                                     names=['subject', 'type'])
data = np.round(np.random.randn(4, 6), 1)
# starting from column 0, every alternate column multiply by 10
data[:, ::2]*=10
# add 37 for every obs
data += 37

health_data = pd.DataFrame(data, index = index, columns = columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  36.5  26.0  36.3  23.0  37.6
     2      28.0  36.5  30.0  35.7  30.0  37.8
2014 1      41.0  35.5  33.0  37.0  38.0  36.4
     2      40.0  36.5  46.0  37.1  24.0  37.6

Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, for example, index the top-level column by the person's name and get a full DataFrame containing just that person's information:

In [66]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      26.0  36.3
     2      30.0  35.7
2014 1      33.0  37.0
     2      46.0  37.1

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

## Indexing and slicing a MultiIndex

Indexing and slicing on a MultiIndex is designed to be intuitive, and it helps if you think about the indices as added dimensions. 

### Multiply indexed Series

In [67]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [68]:
pop['California', 2000]

33871648

The MultiIndex also supports partial indexing, or indexing just one of the levels in the index. The result is another Series, with the lower-level indices maintained:

In [69]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the MultiIndex is sorted (see discussion in Sorted and Unsorted Indices):

In [70]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [72]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Other types of indexing and selection (discussed in Data Indexing and Selection) work as well; for example, selection based on Boolean masks:

In [73]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [75]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed DataFrame behaves in a similar manner. Consider our toy medical DataFrame from before:

In [76]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  36.5  26.0  36.3  23.0  37.6
     2      28.0  36.5  30.0  35.7  30.0  37.8
2014 1      41.0  35.5  33.0  37.0  38.0  36.4
     2      40.0  36.5  46.0  37.1  24.0  37.6

Remember that columns are primary in a DataFrame, and the syntax used for multiply indexed Series applies to the **columns**.

In [77]:
health_data['Guido', 'HR']

year  visit
2013  1        26.0
      2        30.0
2014  1        33.0
      2        46.0
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the loc, iloc

In [83]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      43.0  36.5
     2      28.0  36.5

With multiply indexed DataFrame, the iloc positions are still the same. e.g. row 0:2 is from row 0 to row 1

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in loc or iloc can be passed a tuple of multiple indices.

In [85]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        43.0
      2        28.0
2014  1        41.0
      2        40.0
Name: (Bob, HR), dtype: float64

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

In [86]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-86-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in slice() function, but a better way in this context is to use an IndexSlice object, which Pandas provides for precisely this situation. For example:

In [88]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]
# the idx[:, 1] means keep all the years and filter for visit = 1
# same for HR. for multiply indexed cols and rows, there are two indexs

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,43.0,26.0,23.0
2014,1,41.0,33.0,38.0


## Rearranging multi-indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data. There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations. 

### Sorted and unsorted indices

Many of the MultiIndex slicing operations will fail if the index is not sorted. Let's take a look at this here. <br>
We'll start by creating some simple multiply indexed data where the indices are not lexographically sorted:

In [90]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.692273
      2      0.903720
c     1      0.233025
      2      0.791034
b     1      0.501230
      2      0.758837
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [91]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted. For various reasons, partial slices and other similar operations **require the levels in the MultiIndex to be in sorted (i.e., lexographical) order**. Pandas provides a number of convenience routines to perform this type of sorting; examples are the sort_index() and sortlevel() methods of the DataFrame. We'll use the simplest, sort_index(), here:

In [93]:
data = data.sort_index()
data

char  int
a     1      0.692273
      2      0.903720
b     1      0.501230
      2      0.758837
c     1      0.233025
      2      0.791034
dtype: float64

With the index sorted in this way, partial slicing will work as expected:

In [94]:
data['a':'b']

char  int
a     1      0.692273
      2      0.903720
b     1      0.501230
      2      0.758837
dtype: float64

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [98]:
print(pop)

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64


In [96]:
pop.unstack(level = 0)
# when level = 0, the original index 0 pivots to column names
# it's the same as pop.unstack(level = 'state')

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [109]:
pop.unstack(level = 1)
# when level = 0, the original index 1 pivots to column names
# it's the same as pop.unstack(level = 'year')

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of unstack() is stack(), which here can be used to recover the original series:

In [101]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to **turn the index labels into columns**; this can be accomplished with the **reset_index** method. Calling this on the population dictionary will result in a DataFrame with a state and year column holding the information that was formerly in the index. For clarity, we can optionally specify the name of the data for the column representation:

In [103]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to **build a MultiIndex from the column values**. This can be done with the **set_index** method of the DataFrame, which returns a multiply indexed DataFrame:

In [104]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as mean(), sum(), and max(). For hierarchically indexed data, these can be passed a level parameter that controls which subset of the data the aggregate is computed on.

In [105]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  36.5  26.0  36.3  23.0  37.6
     2      28.0  36.5  30.0  35.7  30.0  37.8
2014 1      41.0  35.5  33.0  37.0  38.0  36.4
     2      40.0  36.5  46.0  37.1  24.0  37.6

In [106]:
data_mean = health_data.mean(level = 'year')
data_mean

subject   Bob       Guido          Sue      
type       HR  Temp    HR   Temp    HR  Temp
year                                        
2013     35.5  36.5  28.0  36.00  26.5  37.7
2014     40.5  36.0  39.5  37.05  31.0  37.0

By further making use of the **axis** keyword, we can take the mean among levels on the **columns** as well:

In [111]:
data_mean.mean(axis=1, level='type')
# group by column with type being the new columns

type,HR,Temp
year,,
2013,30.0,36.733333
2014,37.0,36.683333


Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year. This syntax is actually a short cut to the GroupBy functionality, which we will discuss in Aggregation and Grouping.